In [4]:
import sys
sys.path.append('../')
import game
import layout as lay
import capture
import environment
import captureGraphicsDisplay
import stable_baselines3 as sb3

rules = capture.CaptureRules()

In [5]:
# Runs an entire game, you can use this for reference/understanding/debugging purposes.
# Other than that, it has no use.
def run_game():
    layout = lay.get_layout('../layouts/defaultCapture.lay')

    rl_agents = capture.load_agents(True, '../baselineTeam', [])
    enemy_agents = capture.load_agents(True, '../baselineTeam', [])

    red_agents = enemy_agents
    blue_agents = rl_agents
    agents = sum([list(el) for el in zip(red_agents, blue_agents)], [])
    game = rules.new_game(layout = layout, agents = agents, display = textDisplay.NullGraphics(), length = 1200, mute_agents = False, catch_exceptions = False)
    agent_index = game.starting_index
    num_agents = 4

    states_explored = 0
    while not game.game_over:
        agent = agents[agent_index]
        
        observation = agent.observation_function(game.state.deep_copy())
        action = agent.get_action(observation)
        # print(game.state)
        states_explored += 1
        # print(states_explored)
        game.state = game.state.generate_successor(agent_index, action)
        game.move_history.append((agent_index, action))
        
        rules.process(game.state, game)
        agent_index = (agent_index + 1) % num_agents

In [6]:
env = environment.PacmanEnv(rules, max_episode_length = 1200)

rl = sb3.DQN('CnnPolicy', env, tensorboard_log = './logs/', verbose = 1, learning_starts = 1000, buffer_size = 10_000, exploration_fraction = 0.1, exploration_final_eps = 0.02, target_update_interval = 1000, learning_rate = 0.00025, gamma = 0.99)

rl.learn(total_timesteps= 100_000)
# rl.pre

Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Blue team starts
We are team BLUE
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Blue team starts
We are team BLUE
Logging to ./logs/DQN_3


Time is up.
Tie game!
Red team starts
We are team BLUE
The Red team has returned at least 18.0 of the opponents' dots.
Blue team starts
We are team BLUE
The Red team has returned at least 18.0 of the opponents' dots.
Blue team starts
We are team BLUE
The Red team has returned at least 18.0 of the opponents' dots.
Red team starts
We are team BLUE
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 384      |
|    ep_rew_mean      | -13.5    |
|    exploration_rate | 0.849    |
| time/               |          |
|    episodes         | 4        |
|    fps              | 388      |
|    time_elapsed     | 3        |
|    total_timesteps  | 1536     |
| train/              |          |
|    learning_rate    | 0.00025  |
|    loss             | 5.3e-06  |
|    n_updates        | 133      |
----------------------------------
The Red team has returned at least 18.0 of the opponents' dots.
Red team starts
We are team BLUE
The Red team has returned at l

In [7]:
layout = lay.get_layout('../layouts/defaultCapture.lay')

rl_agents = capture.load_agents(False, 'myTeamTest', [])
enemy_agents = capture.load_agents(True, '../baselineTeam', [])

rl_agents[0].rl = rl
rl_agents[1].rl = rl

red_agents = enemy_agents
blue_agents = rl_agents
agents = sum([list(el) for el in zip(red_agents, blue_agents)], [])

display = captureGraphicsDisplay.PacmanGraphics(
    redTeam = 'myTeamTest',
    redName = 'Red',
    blueTeam = '../baselineTeam',
    blueName = 'Blue',
    frameTime = 0
    )
NUM_ACTIONS = 4
game = rules.new_game(layout = layout, agents = agents, display = display, length = 1200, mute_agents = False, catch_exceptions = False)
game.run()

Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/rl-pacman/myTeamTest.py
	Arguments: {}
Loading agent team: /home/adrian/facultate/emai/AS/pacman-agent/pacman-contest/src/contest/baselineTeam.py
	Arguments: {}
Blue team starts
The Red team has returned at least 18.0 of the opponents' dots.
